In [ ]:
# Jupyter notebook cell 1: Imports and Detector Class
import cv2
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets

class CurvedLaneDetector:
    def __init__(self):
        self.src = np.float32([[580, 460], [700, 460], [1100, 720], [200, 720]])
        self.dst = np.float32([[300, 0], [1000, 0], [1000, 720], [300, 720]])

    def perspective_transform(self, image):
        M = cv2.getPerspectiveTransform(self.src, self.dst)
        return cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))

    def inverse_perspective_transform(self, image):
        Minv = cv2.getPerspectiveTransform(self.dst, self.src)
        return cv2.warpPerspective(image, Minv, (image.shape[1], image.shape[0]))

    def threshold_binary(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (5, 5), 0)
        _, binary = cv2.threshold(blur, 160, 255, cv2.THRESH_BINARY)
        return binary

    def find_lane_pixels(self, binary_warped):
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:, :], axis=0)
        midpoint = np.int32(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        nwindows = 9
        window_height = np.int32(binary_warped.shape[0]//nwindows)
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        leftx_current = leftx_base
        rightx_current = rightx_base
        margin = 100
        minpix = 50
        left_lane_inds = []
        right_lane_inds = []

        for window in range(nwindows):
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin

            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                              (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                               (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)

            if len(good_left_inds) > minpix:
                leftx_current = np.int32(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:
                rightx_current = np.int32(np.mean(nonzerox[good_right_inds]))

        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds]
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        return leftx, lefty, rightx, righty

    def fit_polynomial(self, binary_warped):
        leftx, lefty, rightx, righty = self.find_lane_pixels(binary_warped)

        if len(leftx) == 0 or len(rightx) == 0:
            return np.zeros_like(binary_warped)

        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        lane_img = np.zeros((binary_warped.shape[0], binary_warped.shape[1], 3), dtype=np.uint8)

        for i in range(len(ploty)-1):
            cv2.line(lane_img,
                     (int(left_fitx[i]), int(ploty[i])),
                     (int(left_fitx[i+1]), int(ploty[i+1])),
                     (255, 255, 0), 10)
            cv2.line(lane_img,
                     (int(right_fitx[i]), int(ploty[i])),
                     (int(right_fitx[i+1]), int(ploty[i+1])),
                     (0, 255, 255), 10)

        return lane_img

    def process_frame(self, frame):
        binary = self.threshold_binary(frame)
        warped = self.perspective_transform(binary)
        lane_img = self.fit_polynomial(warped)
        unwarped = self.inverse_perspective_transform(lane_img)
        result = cv2.addWeighted(frame, 1.0, unwarped, 0.6, 0)
        return result


In [ ]:
# Jupyter notebook cell 2: Video Controls + Execution
video_source_widget = widgets.Dropdown(
    options=['Webcam', 'Video File'],
    value='Webcam',
    description='Source:',
)

video_path_widget = widgets.Text(
    value='lane.mp4',
    description='Video Path:',
    layout=widgets.Layout(width='50%')
)

start_button = widgets.ToggleButton(
    value=False,
    description='▶ Start',
    button_style='success'
)

display(video_source_widget, video_path_widget, start_button)


In [ ]:
# Jupyter notebook cell 3: Main loop
detector = CurvedLaneDetector()

def run_lane_detection():
    source = 0 if video_source_widget.value == 'Webcam' else video_path_widget.value
    cap = cv2.VideoCapture(source)
    try:
        while start_button.value:
            ret, frame = cap.read()
            if not ret:
                break
            frame = cv2.resize(frame, (1280, 720))
            result = detector.process_frame(frame)

            result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
            plt.figure(figsize=(10, 6))
            plt.imshow(result_rgb)
            plt.axis('off')
            clear_output(wait=True)
            display(plt.gcf())

    finally:
        cap.release()
        cv2.destroyAllWindows()

start_button.observe(lambda change: run_lane_detection() if change.new else None, names='value')
